# BioExplorer - Metabolism - Combine movies into gallery view
![](../bioexplorer_metabolism_banner.png)

In [ ]:
from glob import glob
import os
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

movies_folder = os.getenv('METABOLISM_MOVIE_FOLDER')

## Media production

In [ ]:
metabolites = [
    'ATP', 'cAMP', 'O2', 'Glc', 'PCr', 'G1P', 'G6P', 'F6P', 'FBP', 'NADH', 'DHAP', 'Pyr', 'Lac',
    'bHB',  'NADPH', 'E4P', 'MAL', 'AKG', 'CIT', 'GLUT', 'GLN', 'QH2', 'CytCred', 'SUCCOA', 'CoA']

### Create movies from frames

In [ ]:
movie_files = list()
for metabolite in tqdm(metabolites):
    folder = os.path.join(movies_folder, metabolite)
    movie_filename = os.path.join(movies_folder, metabolite + '.mp4')
    movie_files.append(movie_filename)
    cmd = 'ffmpeg -f Image2 -i %s/%%*.png -b:v 16M -threads 16 -y %s 2>/dev/null' % (folder, movie_filename)
    os.system(cmd)

### Combine movies into mosaic

In [ ]:
all_movie_files = glob(os.path.join(movies_folder, '*.mp4'))
movie_files=all_movie_files
print('Found %d movies' % len(movie_files))

In [ ]:
nb_columns = 5
nb_lines = 5
resolution=[3840, 2160]
image_size=[int(3840 / nb_columns), int(2160 / nb_lines)]
print('%dx%d=%d (%dx%d)' % (nb_columns, nb_lines, nb_lines*nb_columns, image_size[0], image_size[1]))

### Legend

In [ ]:
def add_text(draw, x_left, y_top, text, font_size, color):
    font = ImageFont.truetype('Roboto-Bold.ttf', size=font_size)
    draw.text((x_left,y_top), text, font=font, fill=color)

img_overlay = Image.new('RGBA', resolution, (0, 0, 0, 0)) 
draw_overlay = ImageDraw.Draw(img_overlay) 

i = 0
fontsize = 45
color = (255,255,255, 255)
for y in range(nb_lines):
    for x in range(nb_columns):
        x_left = x * image_size[0] + 15
        y_top = y * image_size[1] + 15
        text = metabolites[i]
        i += 1

        add_text(draw_overlay, x_left, y_top, text, fontsize, color)

img_overlay.save(os.path.join(movies_folder, 'mosaic', 'metabolites_legend.png'))

### Background

In [ ]:
img_overlay = Image.new('RGBA', resolution, (0, 0, 0, 0)) 
draw_overlay = ImageDraw.Draw(img_overlay) 

img_background = Image.open(os.path.join(movies_folder, 'metabolism_bioexplorer_4k.png'))
img_background = img_background.resize(image_size)

for y in range(nb_lines):
    for x in range(nb_columns):
        x_left = x * image_size[0]
        y_top = y * image_size[1]
        img_overlay.paste(img_background, [x_left, y_top])

img_overlay.save(os.path.join(movies_folder, 'mosaic', 'metabolites_background.png'))

### Movie mosaic

In [ ]:
def generate_mosaic_frames(output_filename):
    import subprocess
    nb_movies = len(movie_files)

    values = list()
    values.append('ffmpeg')

    for movie_file in movie_files:
        values.append('-i')
        values.append(movie_file)

    values.append('-filter_complex')
    filter_complex = 'nullsrc=size=%dx%d [base];' % (resolution[0], resolution[1])
    for i in range(nb_movies):
        filter_complex += ' [%d:v] setpts=PTS-STARTPTS, scale=%dx%d [vid%d];' % (i, image_size[0], image_size[1], i)

    for i in range(nb_movies):
        if i == 0:
            filter_complex += ' [base]'
        else:
            filter_complex += ' [tmp%d]' % (i - 1)
        filter_complex += '[vid%d] overlay=shortest=1:x=%d:y=%d' % (i, int(i % nb_columns) * image_size[0], (int(i / nb_columns) * image_size[1]))
        if i != nb_movies-1:
            filter_complex += ' [tmp%d];' % i

    values.append('"%s"' % filter_complex)
    values.append('-threads')
    values.append('16')
    values.append(output_filename)
    
    cmd = ''
    for value in values:
        cmd += value + ' '
    os.system(cmd + ' 2>/dev/null')

mosaic_output_folder = os.path.join(movies_folder, 'mosaic')
concentrations_output_folder = os.path.join(mosaic_output_folder, 'concentrations')
if not os.path.exists(concentrations_output_folder):
    os.makedirs(concentrations_output_folder)

concentrations_output_filename = os.path.join(concentrations_output_folder, '%05d.png')
generate_mosaic_frames(concentrations_output_filename)